In [356]:
import pandas as pd
import numpy as np
from datetime import datetime as dt

# displaying options for data frames
pd.set_option("display.max_columns", 5400)
pd.set_option("display.max_colwidth", 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [357]:
bike_data = pd.read_csv("../data_clean/london_merged_clean.csv", encoding = "ISO-8859-1")
terrorism_data = pd.read_csv("../data_clean/global_terrorism_clean_uk_only.csv" , encoding = 'utf-8') #"ISO-8859-1")

In [358]:
terrorism_data = terrorism_data.drop(["Unnamed: 0"], axis=1)

In [359]:
df = bike_data.merge(terrorism_data, how = 'left', on = 'date')

In [360]:
df["event_id"]  = df["event_id"].replace(np.nan, 'NAN')

In [361]:
df["event_id"] = np.where(df["event_id"] != "NAN", True, False)

In [362]:
df.rename(columns = {"event_id" : "attack(y/n)"}, inplace = True)

In [363]:
df['date'] = pd.to_datetime(df['date'])

In [364]:
df = df.set_index("date")

In [365]:
df_base = df[[ "bike_rents", 'attack(y/n)']].copy()

In [366]:
df_base

# dropping duplicate date index rows
df_base = df_base.loc[~df_base.index.duplicated(keep='first')]

In [367]:
def get_dates_of_attack_lst(df):
    dates_of_attack = list(df.loc[df["attack(y/n)"] == True].index)
    return list(dict.fromkeys(dates_of_attack))
    

In [368]:
def get_date_index(date, days = 8):
    date_indexes = []
    week_before = list(pd.date_range(end = date, periods = days))
    week_after = list(pd.date_range(start = date, periods = days))
    for entry in week_before:
        date_indexes.append(entry)
    for entry in week_after:
        date_indexes.append(entry)
    return pd.to_datetime(list(dict.fromkeys(date_indexes)))

In [369]:
def get_rents_counts(df, date_index, days):
    time_ser = get_date_index(date_index , days)
    return df['bike_rents'][time_ser]

In [370]:
def get_diff(ser):
    chunk_size = len(ser)//2
    before_sum = sum(ser[0:chunk_size])
    after_sum =  sum(ser[(chunk_size+1):len(ser)])
    return after_sum - before_sum 

In [371]:
def get_diff_df(df, days = 8):
    lst_diff = [] 
    # get list of terrorism attack dates from df
    lst_of_attacks = get_dates_of_attack_lst(df)
    # for each attack date get list of surrounding dates depending on days passed to the function
    # and from that list get a a series of bike rent counts for that time period
    for elem in lst_of_attacks:
        time_series = get_rents_counts(df, elem, days)
        # dealing with missing values at the beginning and the end of the date series by skipping them
        if time_series.isnull().sum() != 0:
            continue
        lst_diff.append(get_diff(time_series))
    return sum(lst_diff)/len(lst_diff)


In [372]:
get_diff_df(df_base, 12)

-226.20118343195267

In [373]:
get_diff_df(df_base, 8)

-1360.7882352941176

In [374]:
get_diff_df(df_base, 4)

-1310.3139534883721

In [375]:
get_diff_df(df_base, 3)

181.27167630057804

In [376]:
get_diff_df(df_base, 2)

504.7586206896552

In [377]:
def get_before_after_tup(ser):
    lst_b_a = []
    chunk_size = len(ser)//2
    lst_b_a.append(sum(ser[0:chunk_size]))
    lst_b_a.append(sum(ser[(chunk_size+1):len(ser)]))
    return tuple(lst_b_a) 

In [378]:
def get_befre_after_df(df, days = 8):
    lst_before_after = [] 
    # get list of terrorism attack dates from df
    lst_of_attacks = get_dates_of_attack_lst(df)
    # for each attack date get list of surrounding dates depending on days passed to the function
    # and from that list get a a series of bike rent counts for that time period
    for elem in lst_of_attacks:
        time_series = get_rents_counts(df, elem, days)
        # dealing with missing values at the beginning and the end of the date series by skipping them
        if time_series.isnull().sum() != 0:
            continue
        lst_before_after.append(get_before_after_tup(time_series))
    return pd.DataFrame(lst_before_after, columns = ["before_sum", "after_sum"])

In [379]:
df_corr = get_befre_after_df(df_base, 3)

In [380]:
df_corr.corr(method="pearson")

,before_sum,after_sum
before_sum,1.000,0.574
after_sum,0.574,1.000


In [381]:
df.head(1)

,bike_rents,temp_actual(C),temp_feeling(C),humidity,wind_speed,weather_condition,holiday(y/n),weekend(y/n),season,attack(y/n),state,city,location,event_summary,multiple_perps(y/n),attack_succeeded(y/n),suicide_attack(y/n),attack_type,target_type,target_subtype,preps_group_name,affiliated(y/n),no_of_perps,people_killed,people_wounded,weapon_type
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-04,9234,2.480,0.650,94.270,7.500,partly cloudy,False,True,winter,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,NaN


## no Northen Ireland

In [382]:
df_no_NI = df[[ "bike_rents", 'attack(y/n)', 'state']].copy()

In [383]:
df_no_NI

,bike_rents,attack(y/n),state
date,,,
2015-01-04,9234,False,NaN
2015-01-05,20372,True,Northern Ireland
2015-01-06,20613,False,NaN
2015-01-07,21064,False,NaN
2015-01-08,15601,True,Northern Ireland
...,...,...,...
2016-12-30,11566,True,Northern Ireland
2016-12-31,11424,False,NaN
2017-01-01,6421,False,NaN


In [384]:
def get_diff_df_1(df, days = 8):
    lst_diff = [] 
    # get list of terrorism attack dates from df
    lst_of_attacks = list(df.loc[(df["attack(y/n)"] == True) & (df["state"] != "Northern Ireland")].index)
    lst_of_attacks = list(dict.fromkeys(lst_of_attacks))
    # for each attack date get list of surrounding dates depending on days passed to the function
    # and from that list get a a series of bike rent counts for that time period
    for elem in lst_of_attacks:
        time_series = get_rents_counts(df, elem, days)
        # dealing with missing values at the beginning and the end of the date series by skipping them
        if time_series.isnull().sum() != 0:
            continue
        lst_diff.append(get_diff(time_series))
    return sum(lst_diff)/len(lst_diff)
df_no_NI = df_no_NI.loc[~df_no_NI.index.duplicated(keep='first')]

In [385]:
get_diff_df_1(df_no_NI, 8)

919.5

In [386]:
get_diff_df_1(df_no_NI, 4)

-1929.423076923077

In [387]:
get_diff_df_1(df_no_NI, 3)

-569.1538461538462

In [388]:
df.head(1)

,bike_rents,temp_actual(C),temp_feeling(C),humidity,wind_speed,weather_condition,holiday(y/n),weekend(y/n),season,attack(y/n),state,city,location,event_summary,multiple_perps(y/n),attack_succeeded(y/n),suicide_attack(y/n),attack_type,target_type,target_subtype,preps_group_name,affiliated(y/n),no_of_perps,people_killed,people_wounded,weapon_type
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-04,9234,2.480,0.650,94.270,7.500,partly cloudy,False,True,winter,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,NaN


In [389]:
test = df.loc[(df["attack(y/n)"] == True) & (df["state"] != "Northern Ireland")].index

In [390]:
len(test)

30

In [391]:
df['weapon_type'].value_counts()

Explosives    86
Incendiary    85
Firearms      31
Melee         12
Other          2
Name: weapon_type, dtype: int64

In [392]:
test1 = df.loc[(df["attack(y/n)"] == True) & (df["state"] != "Northern Ireland") & (df["weather_condition"] != "rain") & (df["weather_condition"] != "snow" ) ].index

In [393]:
len(test1)

26

In [394]:
df["weather_condition"].value_counts()

clear            317
few clouds       173
partly cloudy    171
rain              82
cloudy            31
snow               1
Name: weather_condition, dtype: int64

In [395]:
df.head(1) 

,bike_rents,temp_actual(C),temp_feeling(C),humidity,wind_speed,weather_condition,holiday(y/n),weekend(y/n),season,attack(y/n),state,city,location,event_summary,multiple_perps(y/n),attack_succeeded(y/n),suicide_attack(y/n),attack_type,target_type,target_subtype,preps_group_name,affiliated(y/n),no_of_perps,people_killed,people_wounded,weapon_type
date,,,,,,,,,,,,,,,,,,,,,,,,,,
2015-01-04,9234,2.480,0.650,94.270,7.500,partly cloudy,False,True,winter,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nan,nan,nan,NaN


In [396]:
mean_rent = int(df["bike_rents"].mean())

In [397]:
mean_rent

27186

In [398]:
df_rain = df[df["weather_condition"] == "rain"]

In [399]:
mean_rain = int(df_rain["bike_rents"].mean())

In [400]:
mean_rain

18496

In [401]:
mean_rain - mean_rent

-8690

In [402]:
weather_values = list(dict(df["weather_condition"].value_counts()).keys())

In [403]:
diffrence_from_mean = {} 
for weather_condition in weather_values:
    df_WC = df[df["weather_condition"] == weather_condition].copy()
    diff_WC_from_mean = mean_rent - int(df_WC["bike_rents"].mean())
    print(diff_WC_from_mean)
    diffrence_from_mean[weather_condition] = diff_WC_from_mean

-3224
-1153
2069
8690
4514
15424


In [404]:
diffrence_from_mean

{'clear': -3224,
 'few clouds': -1153,
 'partly cloudy': 2069,
 'rain': 8690,
 'cloudy': 4514,
 'snow': 15424}

In [405]:
df_WC = df[[ "bike_rents", 'attack(y/n)', "weather_condition"]].copy()
df_WC = df_WC.loc[~df_WC.index.duplicated(keep='first')]

In [406]:
df_WC

,bike_rents,attack(y/n),weather_condition
date,,,
2015-01-04,9234,False,partly cloudy
2015-01-05,20372,True,partly cloudy
2015-01-06,20613,False,clear
2015-01-07,21064,False,few clouds
2015-01-08,15601,True,few clouds
...,...,...,...
2016-12-30,11566,True,cloudy
2016-12-31,11424,False,cloudy
2017-01-01,6421,False,rain


In [407]:
#factored_rents = lambda 

In [408]:
df_WC["bike_rents"][4] + diffrence_from_mean[df_WC["weather_condition"][4]]

14448

In [409]:
#df_WC['bike_rents'] = np.where(df.weather_condition, 0, 0)

In [410]:
test = df_WC

In [411]:
df_WC["weather_condition"] = df_WC["weather_condition"].map(diffrence_from_mean)

In [412]:
df_WC = df_WC.assign(WC_factored_rents = lambda x : x["bike_rents"] + x["weather_condition"])

In [422]:
def get_rents_counts_WC_fatored(df, date_index, days):
    time_ser = get_date_index(date_index , days)
    return df['WC_factored_rents'][time_ser]

In [423]:
def get_befre_after_df_WC_fatored(df, days = 8):
    lst_before_after = [] 
    # get list of terrorism attack dates from df
    lst_of_attacks = get_dates_of_attack_lst(df)
    # for each attack date get list of surrounding dates depending on days passed to the function
    # and from that list get a a series of bike rent counts for that time period
    for elem in lst_of_attacks:
        time_series = get_rents_counts_WC_fatored(df, elem, days)
        # dealing with missing values at the beginning and the end of the date series by skipping them
        if time_series.isnull().sum() != 0:
            continue
        lst_before_after.append(get_before_after_tup(time_series))
    return pd.DataFrame(lst_before_after, columns = ["before_sum", "after_sum"])

In [424]:
get_befre_after_df_WC_fatored(df_WC, 8)

,before_sum,after_sum
0,151071,152641
1,156089,149344
2,135135,136436
3,134342,148664
4,139949,156358
...,...,...
165,172313,188355
166,174512,174240
167,184837,159958
168,189144,102379


In [427]:
def get_diff_factored_WC(df, days = 8):
    lst_diff = [] 
    # get list of terrorism attack dates from df
    lst_of_attacks = get_dates_of_attack_lst(df)
    # for each attack date get list of surrounding dates depending on days passed to the function
    # and from that list get a a series of bike rent counts for that time period
    for elem in lst_of_attacks:
        time_series = get_rents_counts_WC_fatored(df, elem, days)
        # dealing with missing values at the beginning and the end of the date series by skipping them
        if time_series.isnull().sum() != 0:
            continue
        lst_diff.append(get_diff(time_series))
    return sum(lst_diff)/len(lst_diff)


In [428]:
get_diff_factored_WC(df_WC, 8)

-638.4529411764706

In [429]:
get_diff_factored_WC(df_WC, 4)

-1032.156976744186

In [430]:
get_diff_factored_WC(df_WC, 3)

-89.99421965317919